In [1]:
print("Hello Ngoc Son")

Hello Ngoc Son


## I. Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
load_dotenv()

True

In [3]:
client = Groq()

In [4]:
generation_chat_history = [
    {
        "role": "system",
        "content": "Bạn là một lập trình viên Python được giao nhiệm vụ tạo mã Python chất lượng cao"
        "Nhiệm vụ của bạn là tạo ra nội dung tốt nhất có thể cho yêu cầu của người dùng. Nếu nhà cung cấp người dùng phê bình"
        "hãy phản hồi bằng một phiên bản sửa đổi của nỗ lực trước đó của bạn."
    }
]

In [ ]:
generation_chat_history.append(
    {
        "role":"user",
        "content":"Tạo code python thực hiện thuật toán merge sort"
    }
)

In [6]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama-3.1-8b-instant"
).choices[0].message.content

In [7]:
generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [8]:
display_markdown(mergesort_code, raw=True)

Đây là một ví dụ về thuật toán merge sort được thực hiện ở Python:

```python
def merge_sort(arr):
    """
    Thuật toán merge sort được sử dụng để sắp xếp mảng từ nhỏ đến lớn.

    :param arr: Mảng cần sắp xếp.
    :type arr: list
    :return: Mảng đã được sắp xếp.
    :rtype: list
    """
    # Nếu mảng có ít nhất hai phần tử thì chia nó thành hai mảng nhỏ hơn.
    if len(arr) <= 1:
        return arr

    # Chia mảng thành hai phần từ giữa.
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Sắp xếp hai mảng nhỏ hơn.
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Kết hợp hai mảng đã được sắp xếp.
    return merge(left_half, right_half)


def merge(left, right):
    """
    Kết hợp hai mảng đã được sắp xếp thành mảng lớn hơn.

    :param left: Mảng trước.
    :type left: list
    :param right: Mảng sau.
    :type right: list
    :return: Mảng đã được sắp xếp.
    :rtype: list
    """
    merged = []
    left_index = 0
    right_index = 0

    # Kết hợp hai mảng.
    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Thêm phần tử còn lại từ mảng trước.
    while left_index < len(left):
        merged.append(left[left_index])
        left_index += 1

    # Thêm phần tử còn lại từ mảng sau.
    while right_index < len(right):
        merged.append(right[right_index])
        right_index += 1

    return merged


# Ví dụ sử dụng:
if __name__ == "__main__":
    arr = [64, 34, 25, 12, 22, 11, 90]
    print("Mảng trước: ", arr)
    sorted_arr = merge_sort(arr)
    print("Mảng sau: ", sorted_arr)
```

Trên đây là một ví dụ cơ bản về việc sử dụng thuật toán merge sort để sắp xếp mảng số từ nhỏ đến lớn.

Đầu tiên, chúng ta chia mảng thành hai phần từ giữa bằng cách sử dụng hàm `merge_sort`. Sau đó, chúng ta sắp xếp hai mảng nhỏ hơn bằng cách sử dụng hàm `merge_sort` lại một lần nữa cho đến khi mảng có ít nhất hai phần tử. Cuối cùng, chúng ta kết hợp hai mảng đã được sắp xếp bằng cách sử dụng hàm `merge`.

Khi chạy code trên, mảng ban đầu sẽ được sắp xếp và in ra kết quả.

In [3]:
%cd ..

/home/ngocson/NSN/Project/ai-agents-from-scratch


In [2]:
!pwd

/home/ngocson/NSN/Project/ai-agents-from-scratch/notebooks


In [5]:
from colorama import Fore
from dotenv import load_dotenv
from groq import Groq
from src.agentic_patterns.utils.completions import completions_create
from src.agentic_patterns.utils.completions import build_prompt_structure
from src.agentic_patterns.utils.completions import FixedFirstChatHistory
from src.agentic_patterns.utils.completions import update_chat_history
from src.agentic_patterns.utils.logging import fancy_step_tracker

In [6]:
load_dotenv()

BASE_GENERATION_SYSTEM_PROMPT = """
Nhiệm vụ của bạn là tạo ra nội dung tốt nhất có thể đáp ứng yêu cầu của người dùng.
Nếu người dùng đưa ra phản hồi, hãy gửi lại phiên bản đã chỉnh sửa của bài làm trước đó.
Bạn phải luôn luôn xuất ra nội dung đã được chỉnh sửa.
"""
BASE_REFLECTION_SYSTEM_PROMPT = """
Bạn được giao nhiệm vụ đưa ra nhận xét và đề xuất về nội dung do người dùng tạo ra.
Nếu nội dung của người dùng có lỗi hoặc cần cải thiện, hãy xuất ra danh sách các đề xuất
và nhận xét. Nếu nội dung của người dùng ổn và không có gì cần thay đổi, hãy xuất ra: <OK>
"""

In [7]:
class ReflectionAgent:
    """A class that implements a Reflection Agent, which generates responses and reflects
    on them using the LLM to iteratively improve the interaction. The agent first generates
    responses based on provided prompts and then critiques them in a reflection step.
    Attributes:
        model (str): The model name used for generating and reflecting on responses.
        client (Groq): An instance of the Groq client to interact with the language model.
    """
    def __init__(self, model: str="llama-3.1-8b-instant"):
        self.client = Groq()
        self.model = model

    def _request_completion(
            self,
            history: list,
            verbose: int = 0,
            log_title: str = "COMPLETION",
            log_color: str = "",        
    ) -> str:
        """
        A private method to request a completion from the Groq model.

        Args:
            history (list): A list of message forming the conversation or reflection history
            verbose (int, optional): The verbosity level. Defaults to 0 (no output).
            log_title (str, optional): _description_. Defaults to "COMPLETION".
            log_color (str, optional): _description_. Defaults to "".
        Returns:
            str: The model-generated response.
        """
        output = completions_create(self.client, history, self.model)

        if verbose > 0:
            print(log_color, f"\n\n{log_title}\n\n", output)
        
        return output
    
    def generate(self, generation_history: list, verbose: int = 0) -> str:
        """Generates a response based on the provided generation history using the model.


        Args:
            generation_history (list): A list of messages forming the conversation or generation history.
            verbose (int, optional): The verbosity level, controlling printed output. Defaults to 0.

        Returns:
            str: _description_
        """
        return self._request_completion(
            generation_history, verbose, log_title="GENERATION", log_color=Fore.CYAN
        )
    
    def reflect(self, reflection_history: list, verbose: int=0) -> str:
        """Reflects on the generation history by generating a critique or feedback.

        Args:
            reflection_history (list): A list of messages forming the reflection history, typically based on the previous generation or interaction.
            verbose (int, optional): The verbosity level, controlling printed output. Defaults to 0.

        Returns:
            str: The critique or reflection response from the model.
        """
        return self._request_completion(
            reflection_history, verbose, log_title="REFLECTION", log_color=Fore.GREEN
        )
    
    def run(
            self,
            user_msg: str,
            generation_system_prompt: str="",
            reflection_system_prompt: str="",
            n_steps: int=10,
            verbose: int=0,
    ) -> str:
        """Runs the ReflectionAgent over multiple steps, alternating between generating a response and reflecting on it for the specified number of steps.

        Args:
            user_msg (str): The user message or query that initiates the interaction.
            generation_system_prompt (str, optional): The system prompt for guiding the generation process.
            reflection_system_prompt The system prompt for guiding the reflection process.
            n_steps (int, optional): The number of generate-reflect cycles to perform. Defaults to 3.
            verbose (int, optional): The verbosity level controlling printed output. Defaults to 0.

        Returns:
            str: The final generated response after all cycles are completed
        """
        generation_system_prompt += BASE_GENERATION_SYSTEM_PROMPT
        reflection_system_prompt += BASE_REFLECTION_SYSTEM_PROMPT

        generation_history = FixedFirstChatHistory(
            [
                build_prompt_structure(prompt=generation_system_prompt, role="system"),
                build_prompt_structure(prompt=user_msg, role="user")
            ],
            total_length=3,
        )

        reflection_history = FixedFirstChatHistory(
            [build_prompt_structure(prompt=reflection_system_prompt, role="system")],
            total_length=3,
        )

        for step in range(n_steps):
            if verbose > 0:
                fancy_step_tracker(step, n_steps)

                generation = self.generate(generation_history=generation_history, verbose=verbose)
                update_chat_history(generation_history, generation, "assistant")
                update_chat_history(reflection_history, generation, "user")

                critique = self.reflect(reflection_history, verbose=verbose)

                if "<OK>" in critique:
                    print(
                        Fore.RED,
                        "\n\nStop Sequence fond. Stoping the reflection loop...\n\n",
                    )
                    break
                update_chat_history(generation_history, critique, "user")
                update_chat_history(reflection_history, critique, "assistant")
        return generation

In [8]:
agent = ReflectionAgent(
    model="llama-3.1-8b-instant"
)

result = agent.run(
    user_msg="Tạo code python thực hiện thuật toán merge sort.",
    n_steps=3,
    verbose=1
)

print("\nFINAL ANSWER:\n", result)


STEP 1/3

 

GENERATION

 **Thuật toán Merge Sort**

Merge sort là một thuật toán sắp xếp dữ liệu bằng cách chia nhỏ dữ liệu thành các mảnh nhỏ và sau đó ghép chúng lại theo thứ tự có sẵn.

**Code Python**
```python
def merge_sort(arr):
    """
    Xử lý sắp xếp dữ liệu bằng thuật toán merge sort
    
    Parameters:
    arr (list): Dữ liệu cần sắp xếp
    
    Returns:
    list: Dữ liệu đã được sắp xếp
    """
    # Nếu chiều dài của mảng dưới 2, đã sắp xếp
    if len(arr) < 2:
        return arr
    
    # Khai báo khoảng giữa mảng
    mid = len(arr) // 2
    
    # Chia mảng thành hai mảng nhỏ
    left = arr[:mid]
    right = arr[mid:]
    
    # Sắp xếp hai mảng nhỏ
    left = merge_sort(left)
    right = merge_sort(right)
    
    # Ghép hai mảng đã sắp xếp lại
    return merge(left, right)


def merge(left, right):
    """
    Xử lý ghép hai mảng đã sắp xếp
    
    Parameters:
    left (list): Mảng được sắp xếp đầu tiên
    right (list): Mảng được sắp xếp thứ hai
    
    Retur